Requirements:
- [Get Started](../01-Day_1/02-Get_started.ipynb)
- [Data management](../01-Day_1/04-Data_management.ipynb)

# Deep copy with manual attachment

We already saw how to deal with complex data structures using deep copy with implicit attach behavior in [this notebook](../03-Day_3/03-Deep_copy.ipynb). 

It is also possible to proceed to a bottom-up deep copy, in which you can first copy sub-objects on the accelerator and then attach them to existing children.
With this procedure you will have to attach explicitly the pointers to the children. This can be easily apprehend with the subsequent code.


Here the first copyin will pass the arrays on the device memory and the second will provide the complex datatype. But the pointers of the complex datatype (such as v0.x) will still reference the host structure.
We should thus provide to the compiler the information of the datatype as it should be on the device. This is done by adding an `attach` directive.


Here, the pointer and its target are present on the device. The directive will inform the compiler to replace the host pointer with the corresponding device pointer in device memory. 

The `detach` clause can be use to free the structure memory but is not mandatory.

## Exercise

In [ ]:
%%idrrun -a
//  examples/C/Deep_copy_attach_detach_exercise.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <math.h>
#include <float.h>

typedef struct
{
    double* count;
    double* birth_rate;
    double* death_rate;    
} Population;

void free_pop(Population* pop)
{
    // Add openacc directives
    free(pop->count);
    free(pop->birth_rate);
    free(pop->death_rate);
    free(pop);
}

void derivee(double* x, Population* pop, double* dx)
{
    // Add openacc directives
    dx[0] =  pop->birth_rate[0]*x[0] - pop->death_rate[0]*x[0]*x[1];
    dx[1] = -pop->death_rate[1]*x[1] + pop->birth_rate[1]*x[0]*x[1];       
}

void rk4(Population* pop, double dt)
{
        double* x_temp = (double*) malloc(2*sizeof(double));
        double* k1     = (double*) malloc(2*sizeof(double));
        double* k2     = (double*) malloc(2*sizeof(double));
        double* k3     = (double*) malloc(2*sizeof(double));
        double* k4     = (double*) malloc(2*sizeof(double));
        double halfdt  = dt / 2.0;
 
	// Add openacc directives
          for(int i=0; i<2; i++)
             x_temp[i] = pop->count[i];

          derivee(x_temp, pop, k1);
          for(int i=0; i<2; i++)
            x_temp[i] = pop->count[i] + k1[i]*halfdt;
        
          derivee(x_temp, pop, k2);
          for(int i=0; i<2; i++)
            x_temp[i] = pop->count[i] + k2[i]*halfdt;

          derivee(x_temp, pop, k3);
          for(int i=0; i<2; i++)
            x_temp[i] = pop->count[i] + k3[i]*dt;

          derivee(x_temp, pop, k4);
          for(int i=0; i<2; i++)
            pop->count[i] = pop->count[i] + (dt/6.0)*(k1[i] + 2.0*k2[i] + 2.0*k3[i] + k4[i]);
}
                 
    
int main(void)                 
{    
    double ti   =   0.00; 
    double dt   =   0.05;
    double tmax = 100.00; 
    
    Population *pred_prey = (Population *) malloc(sizeof(Population));
    pred_prey->count      = (double*) malloc(2*sizeof(double));
    pred_prey->birth_rate = (double*) malloc(2*sizeof(double));
    pred_prey->death_rate = (double*) malloc(2*sizeof(double));
    
    pred_prey->count[1]      = 15.00;  // predator count
    pred_prey->birth_rate[1] = 0.01;   // predator birth rate
    pred_prey->death_rate[1] = 1.0;    // predator death rate

    pred_prey->count[0]      = 100.00; // prey count
    pred_prey->death_rate[0] = 2.0;    // prey birth rate 
    pred_prey->death_rate[0] = 0.02;   // prey death rate

    // Add openacc directives
    while (ti < tmax)
    {
        ti += dt;
        rk4(pred_prey, dt);
        for(int i=0; i<2; i++)
	    // Add openacc directives
        fprintf(stderr, "%lf %lf %lf\n", ti, pred_prey->count[0], pred_prey->count[1]);
    }   
    free_pop(pred_prey);
    return 0;
}

## Solution

In [ ]:
%%idrrun -a
//  examples/C/Deep_copy_attach_detach_solution.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <math.h>
#include <float.h>

typedef struct
{
    double* count;
    double* birth_rate;
    double* death_rate;    
} Population;

void free_pop(Population* pop)
{
    #pragma acc exit data detach(pred_prey->count, pred_prey->birth_rate, pred_prey->death_rate)
    #pragma acc exit data delete(pred_prey->count, pred_prey->birth_rate, pred_prey->death_rate)    
    #pragma acc exit data delete(pred_prey)
    free(pop->count);
    free(pop->birth_rate);
    free(pop->death_rate);
    free(pop);
}

void derivee(double* x, Population* pop, double* dx)
{
    #pragma acc serial present(pop, pop->birth_rate[0:2], pop->death_rate[0:2], x[0:2], dx[0:2])
    {
    dx[0] =  pop->birth_rate[0]*x[0] - pop->death_rate[0]*x[0]*x[1];
    dx[1] = -pop->death_rate[1]*x[1] + pop->birth_rate[1]*x[0]*x[1];       
    }
}

void rk4(Population* pop, double dt)
{
        double* x_temp = (double*) malloc(2*sizeof(double));
        double* k1     = (double*) malloc(2*sizeof(double));
        double* k2     = (double*) malloc(2*sizeof(double));
        double* k3     = (double*) malloc(2*sizeof(double));
        double* k4     = (double*) malloc(2*sizeof(double));
        double halfdt  = dt / 2.0;
 
        # pragma acc data create(k1[0:2], k2[0:2], k3[0:2], k4[0:2], x_temp[0:2]) present(pop, pop->birth_rate[0:2], pop->death_rate[0:2], pop->count[0:2])
        {

          #pragma acc parallel loop
          for(int i=0; i<2; i++)
             x_temp[i] = pop->count[i];

          derivee(x_temp, pop, k1);
          #pragma acc parallel loop
          for(int i=0; i<2; i++)
            x_temp[i] = pop->count[i] + k1[i]*halfdt;
        
          derivee(x_temp, pop, k2);
          #pragma acc parallel loop
          for(int i=0; i<2; i++)
            x_temp[i] = pop->count[i] + k2[i]*halfdt;

          derivee(x_temp, pop, k3);
          #pragma acc parallel loop
          for(int i=0; i<2; i++)
            x_temp[i] = pop->count[i] + k3[i]*dt;

          derivee(x_temp, pop, k4);
          #pragma acc parallel loop
          for(int i=0; i<2; i++)
            pop->count[i] = pop->count[i] + (dt/6.0)*(k1[i] + 2.0*k2[i] + 2.0*k3[i] + k4[i]);
        }
}
                 
    
int main(void)                 
{    
    double ti   =   0.00; 
    double dt   =   0.05;
    double tmax = 100.00; 
    
    Population *pred_prey = (Population *) malloc(sizeof(Population));
    pred_prey->count      = (double*) malloc(2*sizeof(double));
    pred_prey->birth_rate = (double*) malloc(2*sizeof(double));
    pred_prey->death_rate = (double*) malloc(2*sizeof(double));
    
    pred_prey->count[1]      = 15.00;  // predator count
    pred_prey->birth_rate[1] = 0.01;   // predator birth rate
    pred_prey->death_rate[1] = 1.0;    // predator death rate

    pred_prey->count[0]      = 100.00; //prey count
    pred_prey->death_rate[0] = 2.0;    // prey birth rate 
    pred_prey->death_rate[0] = 0.02;   // prey death rate

    #pragma acc enter data copyin(pred_prey->count[0:2], pred_prey->birth_rate[0:2], pred_prey->death_rate[0:2])    
    #pragma acc enter data copyin(pred_prey) attach(pred_prey->count, pred_prey->birth_rate, pred_prey->death_rate)
    while (ti < tmax)
    {
        ti += dt;
        rk4(pred_prey, dt);
        for(int i=0; i<2; i++)
            #pragma acc update self(pred_prey->count[i:1])
        fprintf(stderr, "%lf %lf %lf\n", ti, pred_prey->count[0], pred_prey->count[1]);
    }   
    free_pop(pred_prey);    
    return 0;
}